This Jupyter notebook is written to produce some cmputations
shown in the paper and generate the plots. It is written using
Julia v1.6.

# Setup packages

In [ ]:
include("setup.jl")

# RK22 series examples

This section of code reproduces the results in Sections 2.1.1 and 2.3.1 of the paper.

In [ ]:
α = sp.symbols("α", real=true)
μ = sp.symbols("μ", real=true)
A = [0 0; 1/(2*α) 0]; b = [1-α, α]; c = [0, 1/(2*α)]
A2 = [0 0; 1/(2*μ) 0]; b2 = [1-μ, μ]; c2 = [0, 1/(2*μ)]

coeffs = bseries(A,b,c,3)
coeffs2 = bseries(A2,b2,c2,3)

latexify(coeffs, cdot=false)

In [ ]:
comp = compose(coeffs,coeffs2,normalize_stepsize=true)
latexify(comp, cdot=false)

In [ ]:
println(latexify(comp, cdot=false))

# Modified equations (Lotka-Volterra)

This reproduces the example in Section 2.4.2.

In [ ]:
function f(du, u, params, t)
  p, q = u
  dp = (2 - q) * p
  dq = (p - 1) * q
  du[1] = dp; du[2] = dq
  return nothing
end

u0 = [1.5, 2.25]
tspan = (0.0, 15.0)
ode = ODEProblem(f, u0, tspan)

dt = 0.1
sol_euler = solve(ode, Euler(), dt=dt)
sol_ref = solve(ode, Tsit5(),saveat=0:0.1:15)

In [ ]:
plt.rc("lines", linewidth=2.5, markersize=5, markeredgewidth=2.5)
plt.rc("legend", loc="best", fontsize="small", fancybox=true, framealpha=1.0)

In [ ]:
u = sol_ref.u
plt.plot(getindex.(u,2),getindex.(u,1),color="k")
plt.scatter(last.(sol_euler.u),first.(sol_euler.u),color="#56B4E9")
plt.legend(["Reference solution",L"Explicit Euler, $\Delta t = 0.1$"]);
plt.xlabel(L"$q$"); plt.ylabel(L"$p$");
#plt.savefig("test.pdf")

In [ ]:
function solve_modified_equation(ode, truncation_orders, dt)
  # Explicit Euler method
  A = @SArray [0//1;]
  b = @SArray [1//1]
  c = @SArray [0//1]

  # Setup of symbolic variables
  @variables dt_sym
  u_sym = @variables p q
  f_sym = similar(u_sym); f(f_sym, u_sym, nothing, nothing)

  sol_euler = solve(ode, Euler(), dt=dt)

  plt.scatter(last.(sol_euler.u), first.(sol_euler.u),
              label="Explicit Euler, dt = $dt", color="k")

  for truncation_order in truncation_orders
    series = modified_equation(f_sym, u_sym, dt_sym, A, b, c, truncation_order)
    series = Symbolics.substitute.(series, dt_sym => dt)
    modified_f, _ = build_function(series, u_sym, expression=Val(false))
    modified_ode = ODEProblem((u, params, t) -> modified_f(u), ode.u0, tspan)
    modified_sol = solve(modified_ode, Tsit5(), abstol=1.0e-6, reltol=1.0e-6)
    fig = plt.plot(last.(modified_sol.u), first.(modified_sol.u),
                    label="Modified ODE, order $(truncation_order-1)")
  end
  plt.legend()
  plt.xlabel(L"$q$"); plt.ylabel(L"$p$");
end

fig = solve_modified_equation(ode, 2, dt)
plt.savefig("../figures/LV1.pdf",bbox_inches="tight")

In [ ]:
fig = solve_modified_equation(ode, 2:3, 0.11)
plt.xlim(0,8)
plt.savefig("../figures/LV2.pdf",bbox_inches="tight")

In [ ]:
fig = solve_modified_equation(ode, 2:4, 0.12)
plt.xlim(0,8)
plt.savefig("../figures/LV3.pdf",bbox_inches="tight")

In [ ]:
u0 = [1., 2.01]
tspan = (0.0, 66.4)
ode = ODEProblem(f, u0, tspan)

dt = 0.1
sol_euler = solve(ode, Euler(), dt=dt)
sol_ref = solve(ode, Tsit5())

# Explicit Euler method
A = @SArray [0//1;]
b = @SArray [1//1]
c = @SArray [0//1]

# Setup of symbolic variables
@variables dt_sym
u_sym = @variables p q
f_sym = similar(u_sym); f(f_sym, u_sym, nothing, nothing)

sol_euler = solve(ode, Euler(), dt=dt)

plt.scatter(last.(sol_euler.u), first.(sol_euler.u), label="Explicit Euler, dt = $dt", color="k")

for truncation_order in 2:6
    series = modified_equation(f_sym, u_sym, dt_sym, A, b, c, truncation_order)
    series = Symbolics.substitute.(series, dt_sym => dt)
    modified_f, _ = build_function(series, u_sym, expression=Val(false))
    modified_ode = ODEProblem((u, params, t) -> modified_f(u), ode.u0, tspan)
    modified_sol = solve(modified_ode, Tsit5(), abstol=1.0e-6, reltol=1.0e-6)
    plt.plot(last.(modified_sol.u), first.(modified_sol.u), label="Modified ODE, order $(truncation_order-1)",alpha=0.8)
end
plt.legend()
plt.xlabel(L"$q$"); plt.ylabel(L"$p$");
plt.savefig("../figures/LV4.pdf",bbox_inches="tight")

# Modifying integrators (Lotka-Volterra again)

This reproduces the example in Section 2.5.

In [ ]:
function f(du, u, params, t)
  p, q = u
  dp = (2 - q) * p
  dq = (p - 1) * q
  du[1] = dp; du[2] = dq
  return nothing
end

u0 = [1.5, 2.25]
tspan = (0.0, 15.0)
ode = ODEProblem(f, u0, tspan)

dt = 0.35
sol_euler = solve(ode, Euler(), dt=dt)
sol_ref = solve(ode, Tsit5(),saveat=0:0.1:15)

In [ ]:
plt.plot(last.(sol_ref.u), first.(sol_ref.u), label="Reference solution", color="k")
plt.scatter(last.(sol_euler.u), first.(sol_euler.u),
             label="Explicit Euler, dt = $dt")
plt.xlim(0.0, 4.0); plt.ylim(0.0, 2.5)
plt.legend()
plt.xlabel(L"$q$"); plt.ylabel(L"$p$");
plt.savefig("../figures/MI_LV1.pdf",bbox_inches="tight")

In [ ]:
# Explicit Euler method
A = @SArray [0//1;]
b = @SArray [1//1]
c = @SArray [0//1]

# Setup of symbolic variables
@variables dt_sym
u_sym = @variables p q
f_sym = similar(u_sym); f(f_sym, u_sym, nothing, nothing)

for truncation_order in 2:4
  series = modifying_integrator(f_sym, u_sym, dt_sym, A, b, c, truncation_order)
  series = Symbolics.substitute.(series, dt_sym => dt)
  modified_f, _ = build_function(series, u_sym, expression=Val(false))
  modified_ode = ODEProblem((u, params, t) -> modified_f(u), ode.u0, tspan)
  modified_sol_euler = solve(modified_ode, Euler(), dt=dt)
  plt.plot(last.(modified_sol_euler.u), first.(modified_sol_euler.u),
        label="Euler, modified ODE order $(truncation_order-1)")
end
plt.xlim(0.0, 4.0), plt.ylim(0.0, 2.5)
plt.legend()
plt.xlabel(L"$q$"); plt.ylabel(L"$p$");
plt.savefig("../figures/MI_LV2.pdf",bbox_inches="tight")

# Modifying integrator for the special RK method and ODE pair

This reproduces the figures in Section 2.6.2.

In [ ]:
# Baseline ODE
function f_baseline(du, u, params, t)
    u1, u2 = u
    norm2 = u1^2 + u2^2
    factor = 1 / norm2
    du[1] = -u2 * factor
    du[2] =  u1 * factor
    return nothing
end

# Modifying integrator equation
function f_modified(du, u, params, t)
    u1, u2 = u
    norm2 = u1^2 + u2^2
    z = h() / norm2
    g6 = 1 + z^2 / 12 + z^4 / 20 + 127 * z^6 / 2016
    factor_f = g6 / norm2
    factor_u = h()^5 / (48 * norm2^6) + 31 * h()^7 / (640 * norm2^8)
    du[1] = -u2 * factor_f + u1 * factor_u
    du[2] =  u1 * factor_f + u2 * factor_u
    return nothing
end

# Analytical solution
u_ana(t) = [cos(t), sin(t)]

energy(u) = sum(abs2, u)
error_l2(u, t) = sqrt(sum(abs2, u - u_ana(t)))

tspan = (0.0, 200.0)
ode_baseline = ODEProblem(f_baseline, u_ana(0.0), tspan)
ode_modified = ODEProblem(f_modified, u_ana(0.0), tspan)

# Step size
h() = 0.6
sol_baseline = solve(ode_baseline, Midpoint(), adaptive=false, dt=h())
sol_modified = solve(ode_modified, Midpoint(), adaptive=false, dt=h())

plt.close("all")
plt.plot(sol_baseline.t, error_l2.(sol_baseline.u, sol_baseline.t),
         label="Baseline")
plt.plot(sol_modified.t, error_l2.(sol_modified.u, sol_modified.t),
         label="Modified")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.xlabel("Time \$t\$"); plt.ylabel("Error");
plt.savefig("../figures/nonlinear_osc_mod_int_h06_error.pdf",
    bbox_inches="tight")

plt.figure()
plt.plot(sol_baseline.t, energy.(sol_baseline.u),
         label="Baseline")
plt.plot(sol_modified.t, energy.(sol_modified.u),
         label="Modified")
plt.xscale("log")
# plt.yscale("log")
plt.legend()
plt.xlabel("Time \$t\$"); plt.ylabel("Energy \$\\|\\!\\|y\\|\\!\\|^2\$");
plt.savefig("../figures/nonlinear_osc_mod_int_h06_energy.pdf",
    bbox_inches="tight")

In [ ]:
tspan = (0.0, 1000.0)
ode_baseline = ODEProblem(f_baseline, u_ana(0.0), tspan)
ode_modified = ODEProblem(f_modified, u_ana(0.0), tspan)

# Step size
h() = 0.3
sol_baseline = solve(ode_baseline, Midpoint(), adaptive=false, dt=h())
sol_modified = solve(ode_modified, Midpoint(), adaptive=false, dt=h())

plt.close("all")
plt.plot(sol_baseline.t, error_l2.(sol_baseline.u, sol_baseline.t),
         label="Baseline")
plt.plot(sol_modified.t, error_l2.(sol_modified.u, sol_modified.t),
         label="Modified")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.xlabel("Time \$t\$"); plt.ylabel("Error");
plt.savefig("../figures/nonlinear_osc_mod_int_h03_error.pdf",
    bbox_inches="tight")

plt.figure()
plt.plot(sol_baseline.t, energy.(sol_baseline.u),
         label="Baseline")
plt.plot(sol_modified.t, energy.(sol_modified.u),
         label="Modified")
plt.xscale("log")
# plt.yscale("log")
plt.legend()
plt.xlabel("Time \$t\$"); plt.ylabel("Energy \$\\|\\!\\|y\\|\\!\\|^2\$");
plt.savefig("../figures/nonlinear_osc_mod_int_h03_energy.pdf",
    bbox_inches="tight")